# LSTM. 8 featerow - mag, x, y, depth, dist, distance, plate, days_diff

In [38]:
import numpy as np
import tensorflow as tf
import pandas as pd
import keras
SEED = 1337
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [2]:
ds_train = tf.data.Dataset.load("../data/ds_train/")
ds_val = tf.data.Dataset.load("../data/ds_val/")
ds_test = tf.data.Dataset.load("../data/ds_test/")

2024-01-02 20:37:08.977400: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-02 20:37:09.306149: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-02 20:37:09.306225: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-02 20:37:09.311884: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-02 20:37:09.311966: I tensorflow/compile

In [3]:
ds_train.cardinality().numpy(), ds_val.cardinality().numpy(), ds_test.cardinality().numpy()

(3294316, 465661, 101326)

In [2]:
BATCH_SIZE = 1024
BLOCK_SIZE = 64

In [5]:
def fix_x_and_y(features, context, labels):
    return features, labels

In [6]:
ds_train = ds_train.map(fix_x_and_y)
ds_val = ds_val.map(fix_x_and_y)
ds_test = ds_test.map(fix_x_and_y)

In [7]:
for x, y in ds_train.batch(BATCH_SIZE).take(1):
    print(x.shape)
    print(y.shape)

(1024, 64, 9)
(1024, 1)


In [8]:
# ys = []
# for x, y in ds_train.batch(BATCH_SIZE):
#     ys.append(y.numpy())
# ys = np.concatenate(ys)
# np.bincount(tf.cast(ys, tf.int32).numpy().flatten())

In [9]:
#neg, pos = np.bincount(tf.cast(ys, tf.int32).numpy().flatten())
neg, pos = 3094474, 199842
total = neg + pos
total, pos, neg

(3294316, 199842, 3094474)

In [10]:
ds_train = ds_train.batch(BATCH_SIZE).shuffle(2000, seed=SEED).prefetch(tf.data.experimental.AUTOTUNE)
ds_val = ds_val.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

In [42]:
@tf.keras.saving.register_keras_serializable()
class MyModel(tf.keras.Model):
    def __init__(self, n_embed):
        super(MyModel, self).__init__()
        self.embed_plate = tf.keras.layers.Embedding(64, n_embed // 9)
        self.embed_dd = tf.keras.layers.Embedding(20, n_embed // 9)
        self.embed_magtype = tf.keras.layers.Embedding(20, n_embed // 9)
        self.embed = tf.keras.layers.Dense(n_embed // 9 * 6)
        self.lstm1 = tf.keras.layers.LSTM(n_embed, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed)
        self.dense1 = tf.keras.layers.Dense(n_embed, activation='relu')
        self.dense2 = tf.keras.layers.Dense(1, activation='sigmoid')
        self.conc = tf.keras.layers.Concatenate(axis=-1)

    def call(self, x):
        cont, plate, dd, magtype = x[:,:,:-3], x[:,:,-3], x[:,:,-2], x[:,:,-1] 
        x1 = self.embed(cont)
        x2 = self.embed_plate(plate)
        x3 = self.embed_dd(dd)
        x4 = self.embed_magtype(magtype)
        x = self.conc([x1, x2, x3, x4])
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.dense1(x)
        x = self.dense2(x)
        return x

In [11]:
model = MyModel(135)

In [47]:
model.build(input_shape=(None, BLOCK_SIZE, 9))
model.count_params()

313366

In [48]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
            optimizer='adam',
            metrics=['accuracy', tf.keras.metrics.Recall(), tf.keras.metrics.Precision()])

In [15]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)
class_weight = {0: weight_for_0, 1: weight_for_1}
history = model.fit(ds_train, epochs=20, validation_data=ds_val)

Epoch 1/20


2024-01-02 20:37:31.513321: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 206 of 2000
2024-01-02 20:37:41.520445: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 408 of 2000
2024-01-02 20:38:01.478519: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 798 of 2000
2024-01-02 20:39:04.564275: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 903 of 2000
2024-01-02 20:39:21.481071: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 1029 of 2000
2024-01-02 20:39:31.507497: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a

3218/3218 [==============================] - 925s 160ms/step - loss: 0.1656 - accuracy: 0.9462 - recall: 0.2101 - precision: 0.6838 - val_loss: 0.2090 - val_accuracy: 0.9422 - val_recall: 0.1338 - val_precision: 0.6846
Epoch 2/20


2024-01-02 20:52:51.918833: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 77 of 2000
2024-01-02 20:53:02.033152: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 161 of 2000
2024-01-02 20:53:22.045190: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 356 of 2000
2024-01-02 20:53:32.057698: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 439 of 2000
2024-01-02 20:53:51.941143: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 630 of 2000
2024-01-02 20:54:11.898270: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3218/3218 [==============================] - 752s 176ms/step - loss: 0.1537 - accuracy: 0.9493 - recall: 0.2598 - precision: 0.7316 - val_loss: 0.2024 - val_accuracy: 0.9417 - val_recall: 0.1084 - val_precision: 0.7087
Epoch 3/20


...

3218/3218 [==============================] - 720s 154ms/step - loss: 0.0855 - accuracy: 0.9704 - recall: 0.6033 - precision: 0.8676 - val_loss: 0.3106 - val_accuracy: 0.9314 - val_recall: 0.2717 - val_precision: 0.4208
Epoch 19/20


2024-01-03 00:10:25.037278: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 82 of 2000
2024-01-03 00:10:35.052224: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 165 of 2000
2024-01-03 00:10:55.047214: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 333 of 2000
2024-01-03 00:11:15.023460: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 495 of 2000
2024-01-03 00:11:34.962020: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 662 of 2000
2024-01-03 00:11:44.978751: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3218/3218 [==============================] - 738s 155ms/step - loss: 0.0812 - accuracy: 0.9716 - recall: 0.6197 - precision: 0.8767 - val_loss: 0.3806 - val_accuracy: 0.9212 - val_recall: 0.2727 - val_precision: 0.3364
Epoch 20/20


2024-01-03 00:22:42.998682: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 87 of 2000
2024-01-03 00:22:53.030179: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 174 of 2000
2024-01-03 00:23:12.953189: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 345 of 2000
2024-01-03 00:23:22.967439: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 418 of 2000
2024-01-03 00:23:42.926249: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a while): 607 of 2000
2024-01-03 00:23:52.996707: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:14: Filling up shuffle buffer (this may take a w

3218/3218 [==============================] - 733s 155ms/step - loss: 0.0789 - accuracy: 0.9725 - recall: 0.6321 - precision: 0.8814 - val_loss: 0.3670 - val_accuracy: 0.9236 - val_recall: 0.2290 - val_precision: 0.3341


In [16]:
model.evaluate(ds_test)

99/99 [==============================] - 14s 135ms/step - loss: 0.1054 - accuracy: 0.9702 - recall: 0.2145 - precision: 0.2849


[0.10537658631801605,
 0.9701557159423828,
 0.2145359069108963,
 0.2848837077617645]

In [17]:
model.save("../models/model_1.keras")

In [18]:
# save history
history = pd.DataFrame.from_dict(history.history)
history.to_csv("../models/history_model_v1.csv", index=False)